<a href="https://colab.research.google.com/github/Mike030668/Class_test/blob/master/%D0%A2%D0%B5%D1%81%D1%82_%D0%B2_%D0%BA%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D1%83_%D0%A3%D0%98%D0%98_%D0%BD%D0%B0_%D0%9C%D0%A1%D0%95_%D0%B2%D0%B5%D1%80_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python

In [ ]:
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFont # Модули работы с изображениями
import time

# Загрузка массива контрольных лиц с HTML

In [ ]:
import gdown
file_id = '1n-HZ_8jsYpBH8BJGpDSeLIEKRc3vAhSQ'
gdown.download('https://drive.google.com/uc?id=' + file_id, 'control_faces.h5', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1n-HZ_8jsYpBH8BJGpDSeLIEKRc3vAhSQ
To: /content/control_faces.h5
730MB [00:02, 287MB/s]


'control_faces.h5'

In [ ]:
control_faces = np.load('/content/control_faces.h5')
control_faces.shape

(3714, 128, 128, 3)

# MSE

In [ ]:
grid_size = 20
radius = 5
mag = 1

def get_circle_stencil(radius):
    xx, yy = np.meshgrid(np.linspace(-grid_size/2+1/2, grid_size/2-1/2, grid_size),
                         np.linspace(-grid_size/2+1/2, grid_size/2-1/2, grid_size))
    dist = np.sqrt(xx**2 + yy**2)
    inner = dist < (radius - 1/2)
    return inner.astype(float)

def create_noise(mag, n_dim=2):
    # return np.random.normal(0, mag, size=(grid_size,)*n_dim)
    return np.random.uniform(0, mag, size=(grid_size,)*n_dim)

def create_noisy_sample(alpha, n_dim=2):
    return (np.random.uniform(0, 1-alpha, size=(grid_size,)*n_dim) + 
            alpha*get_circle_stencil(radius))

In [ ]:
noise = create_noise(mag=1, n_dim=2)
noise_b = get_circle_stencil(radius)

In [ ]:
# создадим функцию среднеквадратичной ошибки
def getMSEmasive(x1, x2): 
  return np.square(np.subtract(x1, x2)) # и возвращаем сумму квадратов разницы, делённую на длину разницы

In [ ]:
# создадим функцию среднеквадратичной ошибки
def allMSEmasives(x1, x2): 
  mse = []
  for i in range(x1.shape[0]):
     for j in range(x2.shape[0]):
           mse.append(np.square(np.subtract(x1[i], x2[j])).min())
  
  return np.array(mse).min() # и возвращаем сумму квадратов разницы, делённую на длину разницы

In [ ]:
getMSEmasive(noise, noise_b).mean()

0.30239980048398146

In [ ]:
def norma(*args):
    return [a / np.linalg.norm(a) for a in args]

def cosim(a, b):
    return np.sum(a * b)

def all_mse(a, b):
    return np.sqrt(np.sum((a-b)**2))

def bce(a, b):
    # binary cross entropy implemented from tensorflow / keras
    eps = 1e-7
    res = a * np.log(b + eps)
    res += (1 - a) * np.log(1 - b + eps)
    return np.mean(-res)

In [ ]:
mse(*norma(control_faces[:10], control_faces[10:20]))

0.5512405

In [ ]:
noise = control_faces[:10]# create_noise(mag=1, grid_size=grid_size)
noise_b = control_faces[10:20] #create_noise(mag=1, grid_size=grid_size)
circle = control_faces[20:30] # get_circle_stencil(radius=radius, grid_size=grid_size)
sample = control_faces[30:40] #create_noise(mag=1, grid_size=grid_size) + alpha * circle_reference

print('NoiseA-NoiseB:', mse(*norma(noise, noise_b)))    # 0.718
print('Circle-Circle:', mse(*norma(circle, circle)))    # 0.000
print('Circle-Noise:', mse(*norma(circle, noise)))      # 1.168
print('Noise-Sample:', mse(*norma(noise, sample)))      # 0.697
print('Circle-Sample:', mse(*norma(circle, sample)))    # 1.100

print('NoiseA-NoiseB:', cosim(*norma(noise, noise_b)))  # 0.741
print('Circle-Circle:', cosim(*norma(circle, circle)))  # 1.000
print('Circle-Noise:', cosim(*norma(circle, noise)))    # 0.317
print('Noise-Sample:', cosim(*norma(noise, sample)))    # 0.757
print('Circle-Sample:', cosim(*norma(circle, sample)))  # 0.393

print('NoiseA-NoiseB:', bce(*norma(noise, noise_b)))    # 0.194
print('Circle-Circle:', bce(*norma(circle, circle)))    # 0.057
print('Circle-Noise:', bce(*norma(circle, noise)))      # 0.111
print('Noise-Circle:', bce(*norma(noise, circle)))      # 0.636
print('Noise-Sample:', bce(*norma(noise, sample)))      # 0.192
print('Circle-Sample:', bce(*norma(circle, sample)))    # 0.104

NoiseA-NoiseB: 0.5512405
Circle-Circle: 0.0
Circle-Noise: 0.5314944
Noise-Sample: 0.5674159
Circle-Sample: 0.6323476
NoiseA-NoiseB: 0.8480593
Circle-Circle: 0.9999912
Circle-Noise: 0.858749
Noise-Sample: 0.83902097
Circle-Sample: 0.80006874
NoiseA-NoiseB: 0.009422151
Circle-Circle: 0.009241038
Circle-Noise: 0.009545299
Noise-Circle: 0.009413267
Noise-Sample: 0.009559114
Circle-Sample: 0.009809426


# Загрузка видео с HTML

In [ ]:
file_id = '1ETKBfNoXO8Kafr8Kn1tbzozb94ygRI41'
gdown.download('https://drive.google.com/uc?id=' + file_id, 'test_video.mp4', quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1ETKBfNoXO8Kafr8Kn1tbzozb94ygRI41
To: /content/test_video.mp4
204MB [00:00, 223MB/s]


'test_video.mp4'

# Поиск, обрезание и склейка видео

In [ ]:
import cv2 as cv2

path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier( path )

In [ ]:
path_test_video = '/content/test_video.mp4'
cap = cv2.VideoCapture(path_test_video)

step_frame = 1 #сколько берем кадров
bias = 0.033

start = time.time()
num_frame = []
np_cutfilm = []
cadr = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
   
    if not ret:
      break

    if cadr % step_frame == 0:
      image_base = frame
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      faces = face_cascade.detectMultiScale(
          gray,
          scaleFactor= 1.1,
          minNeighbors= 5,
          minSize=(10, 10)
      )

      # Рисуем квадраты вокруг лиц
      frame_faces = []
      for i,(x, y, w, h) in enumerate(faces):
          #print(i)
          cv2.rectangle(faces, (x, y), (x+w, y+h), (255, 255, 0), 2)

          face_np = image_base[y:y+h,x:x+w,:]

          face_img = Image.fromarray(face_np) #cv2_imshow(face_np) # cv2.imwrite(face_np) #path_save)#
          face_img = face_img.resize((128, 128)) 
          face_np = np.array(face_img)/255.
          frame_faces.append(face_np)

      frame_faces = np.array(frame_faces)
      if frame_faces.shape[0] != 0:
          #print(frame_faces.shape[0])
          #pred_face = train_model.predict(frame_faces)
          #MSEs = getMSEmasive(frame_faces, pred_face)
          #errors = [MSEs[i].mean() for i in range(frame_faces.shape[0])] # посчитаем ошибку для всех изображений
          idx = np.random.choice(range(1200), frame_faces.shape[0])
          errors = []
          for i in range(frame_faces.shape[0]):
             err = getMSEmasive(frame_faces[i], control_faces[idx])
             errors.append(err)
          errors = np.array(errors)
          errors = [errors.mean() for i in range(frame_faces.shape[0])] # посчитаем ошибку для всех изображений
          #print("Средняя ошибка на aliens_data:", np.round(np.sum(errors) / len(errors), 4)) # посчитаем для последних 20 среднюю ошибку
          print("Минимальная ошибка на aliens_data:", np.round(np.min(errors),4)) #найдем минимальную ошибку
          if np.min(errors) < bias:
            print('MSE ', np.round(np.min(errors),4), ' кадр добавлен')
            np_cutfilm.append(image_base)
            num_frame.append(cadr)

    cadr+=1
          
    cv2.waitKey(1) & 0xFF

np_cutfilm = np.array(np_cutfilm)
num_frame = np.array(num_frame)
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print('оставлено кадров', np_cutfilm.shape)


fps_of_video = int(cv2.VideoCapture(path_test_video).get(cv2.CAP_PROP_FPS)/step_frame)
if fps_of_video == 0:
   fps_of_video = 1

time_person = np_cutfilm.shape[0]/fps_of_video
time_person = round(time_person/60,1)
print('время Кевина на видео: ',time_person, ' минут')

finish = time.time() - start
print('время обработки', finish)

height, width = np_cutfilm.shape[1:3]

# Определяем кодек и создаем объект VideoWriter
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter()
output_file_name = '/content/drive/MyDrive/Colab Bases/Команда УИИ/newmse_0.033_video_cut1.mp4' #"output_single.mp4"
out.open(output_file_name, fourcc, fps_of_video, (width, height), True)

for i in range(np_cutfilm.shape[0]):
    out.write((np_cutfilm[i]))


# Высвобождаем ресурсы
cap.release()
out.release()

Минимальная ошибка на aliens_data: 0.056
Минимальная ошибка на aliens_data: 0.0543
Минимальная ошибка на aliens_data: 0.0676
Минимальная ошибка на aliens_data: 0.0551
Минимальная ошибка на aliens_data: 0.056
Минимальная ошибка на aliens_data: 0.0552
Минимальная ошибка на aliens_data: 0.0706
Минимальная ошибка на aliens_data: 0.0517
Минимальная ошибка на aliens_data: 0.0512
Минимальная ошибка на aliens_data: 0.0572
Минимальная ошибка на aliens_data: 0.0719
Минимальная ошибка на aliens_data: 0.0542
Минимальная ошибка на aliens_data: 0.0564
Минимальная ошибка на aliens_data: 0.0526
Минимальная ошибка на aliens_data: 0.0575
Минимальная ошибка на aliens_data: 0.0784
Минимальная ошибка на aliens_data: 0.0524
Минимальная ошибка на aliens_data: 0.0521
Минимальная ошибка на aliens_data: 0.0484
Минимальная ошибка на aliens_data: 0.0739
Минимальная ошибка на aliens_data: 0.0571
Минимальная ошибка на aliens_data: 0.0571
Минимальная ошибка на aliens_data: 0.0533
Минимальная ошибка на aliens_data: 0

In [ ]:
np_cutfilm = np.array(np_cutfilm)
num_frame = np.array(num_frame)
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print('оставлено кадров', np_cutfilm.shape)


fps_of_video = int(cv2.VideoCapture(path_test_video).get(cv2.CAP_PROP_FPS)/step_frame)
if fps_of_video == 0:
   fps_of_video = 1

time_person = np_cutfilm.shape[0]/fps_of_video
time_person = round(time_person/60,1)
print('время Кевина на видео: ',time_person, ' минут')

finish = time.time() - start
print('время обработки', finish)

height, width = np_cutfilm.shape[1:3]

# Определяем кодек и создаем объект VideoWriter
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter()
output_file_name = '/content/drive/MyDrive/Colab Bases/Команда УИИ/newmse_0.05_video_cut25.mp4' #"output_single.mp4"
out.open(output_file_name, fourcc, fps_of_video, (width, height), True)

for i in range(np_cutfilm.shape[0]):
    out.write((np_cutfilm[i]))


# Высвобождаем ресурсы
cap.release()
out.release()

оставлено кадров (125, 408, 720, 3)
время Кевина на видео:  2.1  минут
время обработки 393.35879588127136
